<a href="https://colab.research.google.com/github/NikolayAI/aiohttp/blob/main/08_aiohttp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%pip install aiohttp

In [35]:
import xml.etree.ElementTree as ET
import aiohttp
import asyncio
import nest_asyncio
from datetime import datetime

BASE_INDEX = 20038.52

async def fetch_quote(http_client, quotation):
    exchange_url = f'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{quotation}.xml'
    async with http_client.get(exchange_url) as raw_quotations:
        xml = await raw_quotations.text()
        tree = ET.fromstring(xml)
        for row in tree.findall(".//row"):
            if 'LAST' in row.attrib:
                return float(row.attrib['LAST'])
        return None


async def get_moexbc(quotations_data):
    total_base = sum(data['base_capitalization'] for data in quotations_data.values())

    divider = BASE_INDEX / total_base

    async with aiohttp.ClientSession() as http_client:
        current_caps = []

        p = [fetch_quote(http_client, quotation) for quotation in quotations_data]

        prices = await asyncio.gather(*p)

        for quotation, price in zip(quotations_data, prices):
            data = quotations_data[quotation]
            data['weight'] = data['base_capitalization'] / total_base
            data['last_price'] = price if price else 0.0
            data['current_cap'] = data['base_capitalization'] / data['base_price'] * data['last_price'] if data['base_price'] != 0 else 0
            current_caps.append(data['current_cap'])

        current_caps_total = round(sum(current_caps), 2)

        for i, (quotation, price) in enumerate(zip(quotations_data, prices)):
            data = quotations_data[quotation]
            print_current_caps_total = current_caps_total if i == 0 else ""
            print_divider = round(divider,2) if i == 0 else ""
            new_index = (print_divider * print_current_caps_total) if i == 0 else ""

            if new_index: print(f"Индекс: {new_index}")


async def main():
    quotations_data = {
        'CHMF': {'base_price': 1435.00, 'base_capitalization': 110.595},
        'GAZP': {'base_price': 162.25, 'base_capitalization': 643.752},
        'GMKN': {'base_price': 163.96, 'base_capitalization': 388.556},
        'IRAO': {'base_price': 4.069, 'base_capitalization': 89.209},
        'LKOH': {'base_price': 6911.50, 'base_capitalization': 772.998},
        'MGNT': {'base_price': 7239.00, 'base_capitalization': 109.185},
        'NLMK': {'base_price': 182.90, 'base_capitalization': 92.078},
        'NVTK': {'base_price': 1484.00, 'base_capitalization': 189.247},
        'PLZL': {'base_price': 10875.00, 'base_capitalization': 130.218},
        'ROSN': {'base_price': 600.30, 'base_capitalization': 139.966},
        'RUAL': {'base_price': 34.89, 'base_capitalization': 95.415},
        'SBER': {'base_price': 276.00, 'base_capitalization': 701.917},
        'SNGS': {'base_price': 28.215, 'base_capitalization': 171.632},
        'TATN': {'base_price': 688.20, 'base_capitalization': 321.656},
        'YDEX': {'base_price': 2611.0, 'base_capitalization': 83.5040}
    }

    print("Старт вычислений")

    start = datetime.now()

    await get_moexbc(quotations_data)

    calculation_time = (datetime.now() - start).total_seconds()

    print(f"Вычисления завершены: {calculation_time:.2f} секунд")

if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())

Старт вычислений
Индекс: 16526.968
Вычисления завершены: 0.78 секунд
